In [1]:
import librosa 
import numpy as np 
from scipy.fft import fft, fftfreq, rfft, rfftfreq 
import pandas as pd 
import matplotlib.pyplot as plt 
import math 
import crepe 
import os

In [2]:
SONG_PATH = "D:\\Github\\Music-Rec-RAC-DataSci\\Song DataSet"

# os.listdir(SONG_PATH)

dirs = [f for f in os.listdir(SONG_PATH) if os.path.isdir(os.path.join(SONG_PATH,f))]
dirs

['17さいのうた。 ⧸ 『ユイカ』【MV】',
 '213',
 'Akane-iro_ni_Somaru_Saka_-_Ai_Koko_Kara',
 'Akane-Iro_ni_Somaru_Saka_-_Mezamenai_Wish',
 'AKB0048_-_Niji_no_Ressha',
 'AKB0048_Next_Stage_-_Aruji_Naki_Sono_Koe',
 'Angel_Beats_-_Alchemy',
 'Angel_Beats_-_Crow_Song',
 'Angel_Beats_-_Ichiban_no_Takaramono',
 'Angel_Beats_-_Ichiban_no_Takaramono_Karuta_Version',
 'AnoHana_-_Secret_Base_Kimi_ga_Kureta_Mono_10_Years_After',
 'Aoi_Hana_-_Centifolia',
 'Aoi_Hana_-_Centifolia_2',
 'Bakemonogatari-Kimi no Shiranai Monogatari',
 'Bakemonogatari-Renai Circulation',
 'Bakemonogatari-SugarSweet Nightmare',
 'Bleach_-_Change',
 'Bleach_-_Harukaze',
 'Blood-C_-_Junketsu_Paradox',
 'Chuunibyou_Demo_Koi_ga_Shitai_-_Inside_Identity',
 'Chuunibyou_Demo_Koi_ga_Shitai_Lite_-_Shikkoku_ni_Odoru_Haou_Bushi',
 'Densetsu_no_Yuusha_no_Densetsu_-_Last_Inferno',
 'Ef_-_A_Tale_of_Melodies_-_Ebullient_Future',
 'End_of_Evangelion_-_Thanatos_If_I_Cant_Be_Yours',
 'Evangelion_-_Cruel_Angels_Thesis',
 'Evangelion_Death__Rebirth_-_Tama

In [3]:
music_pure_db = pd.DataFrame(columns=['file', 'time', 'frequency', 'confidence', 'note'])

In [4]:
# # get files in a directory
# dir = dirs[0]
# files = os.path.join(SONG_PATH, dir, "vocals.mp3")
# files

In [5]:
# # load audio file
# y, sr = librosa.load(files)
# time, frequency, confidence, activation = crepe.predict(y, sr, model_capacity='tiny', viterbi=True)

# df = pd.DataFrame({'time': time, 'frequency': frequency, 'confidence': confidence})
# df['file']=dir
# df['note']=df['frequency'].apply(lambda x: librosa.hz_to_note (x))
# df

In [6]:
# music_pure_db = pd.concat([music_pure_db, df], ignore_index=True)
# music_pure_db.sort_values(by=['file', 'time'], inplace=True)
# music_pure_db

In [7]:
# if (df.columns == ['time', 'frequency', 'confidence', 'file']).all():
#     print("True")

In [8]:
music_pure_db = pd.read_parquet('music_pure_db.parquet')

In [9]:
dfs = [music_pure_db]

for dir in dirs:
    file = os.path.join(SONG_PATH, dir, "vocals.mp3")
    if os.path.exists(os.path.join(SONG_PATH, dir, 'data.csv')):
        df = pd.read_csv(os.path.join(SONG_PATH, dir, 'data.csv'))
        df.to_parquet(os.path.join(SONG_PATH, dir, 'data.parquet'), index=False)
        print(f"Reformatted csv to parquet version: {dir}")
        if len(df.columns) == 4:
            if (df.columns == ['time', 'frequency', 'confidence', 'file']).all():
                # print(f"Old version of {dir}")
                df.drop(columns=['file'], inplace=False).to_csv(os.path.join(SONG_PATH, dir, 'data.csv'), index=False) 
                print(f"Reformatted to lastest version: {dir}")
        # remove csv file
        os.remove(os.path.join(SONG_PATH, dir, 'data.csv'))


    if os.path.exists(os.path.join(SONG_PATH, dir, 'data.parquet')):
        df = pd.read_parquet(os.path.join(SONG_PATH, dir, 'data.parquet'))
        df['file']=dir
        
        # verify data
        y, sr = librosa.load(file)
        duration = len(y)/sr
        duration = duration*100 # number of data points
        duration = math.ceil(duration)
        if duration == len(df['time']):
            print(f"Loading: {dir}")
            dfs.append(df.copy())
            continue
        print(
            f"Duration of {dir} is {duration} but data.parquet has {len(df['time'])}")

    print(f"Processing: {dir}")
    
    # load audio file
    y, sr = librosa.load(file)
    time, frequency, confidence, activation = crepe.predict(y, sr, model_capacity='tiny', viterbi=True)

    df = pd.DataFrame({'time': time, 'frequency': frequency, 'confidence': confidence})
    df['file']=dir
    # df['note']=df['frequency'].apply(lambda x: librosa.hz_to_note (x))
    
    dfs.append(df.copy())
    # df.drop(columns=['file'], inplace=False).to_csv(os.path.join(SONG_PATH, dir, 'data.csv'), index=False) #! Obsolete Version 
    df.drop(columns=['file'], inplace=False).to_parquet(
        os.path.join(SONG_PATH, dir, 'data.parquet'), index=False)


music_pure_db = pd.concat(dfs, ignore_index=True)

music_pure_db.sort_values(by=['file', 'time'], inplace=True)
music_pure_db['note']=librosa.hz_to_note(music_pure_db['frequency'])

Loading: 17さいのうた。 ⧸ 『ユイカ』【MV】
Loading: 213
Loading: Akane-iro_ni_Somaru_Saka_-_Ai_Koko_Kara
Loading: Akane-Iro_ni_Somaru_Saka_-_Mezamenai_Wish
Loading: AKB0048_-_Niji_no_Ressha
Loading: AKB0048_Next_Stage_-_Aruji_Naki_Sono_Koe
Loading: Angel_Beats_-_Alchemy
Loading: Angel_Beats_-_Crow_Song
Loading: Angel_Beats_-_Ichiban_no_Takaramono
Loading: Angel_Beats_-_Ichiban_no_Takaramono_Karuta_Version
Loading: AnoHana_-_Secret_Base_Kimi_ga_Kureta_Mono_10_Years_After
Loading: Aoi_Hana_-_Centifolia
Loading: Aoi_Hana_-_Centifolia_2
Loading: Bakemonogatari-Kimi no Shiranai Monogatari
Loading: Bakemonogatari-Renai Circulation
Loading: Bakemonogatari-SugarSweet Nightmare
Loading: Bleach_-_Change
Loading: Bleach_-_Harukaze
Loading: Blood-C_-_Junketsu_Paradox
Loading: Chuunibyou_Demo_Koi_ga_Shitai_-_Inside_Identity
Loading: Chuunibyou_Demo_Koi_ga_Shitai_Lite_-_Shikkoku_ni_Odoru_Haou_Bushi
Loading: Densetsu_no_Yuusha_no_Densetsu_-_Last_Inferno
Loading: Ef_-_A_Tale_of_Melodies_-_Ebullient_Future
Loading:

In [10]:
music_pure_db.to_csv('music_pure_db.csv', index=False)

music_pure_db.to_parquet('music_pure_db.parquet', compression='Brotli', index=False)

In [8]:
# load data 
# music_pure_db = pd.read_csv('music_pure_db.csv')

music_pure_db = pd.read_parquet('music_pure_db.parquet')

In [ ]:
DB_music = pd.read_csv('DB_music.csv')

In [11]:
DB_music = pd.DataFrame(columns=['Name', 'Album', 'Artist', 'Genre', 'Subgenre', 'Publish Date',
       'Duration', 'BPM', 'Vocal Rage', 'E2_binary', 'F2_binary', 'F♯2_binary',
       'G2_binary', 'G♯2_binary', 'A2_binary', 'A♯2_binary', 'B2_binary',
       'C3_binary', 'C♯3_binary', 'D3_binary', 'D♯3_binary', 'E3_binary',
       'F3_binary', 'F♯3_binary', 'G3_binary', 'G♯3_binary', 'A3_binary',
       'A♯3_binary', 'B3_binary', 'C4_binary', 'C♯4_binary', 'D4_binary',
       'D♯4_binary', 'E4_binary', 'F4_binary', 'F♯4_binary', 'G4_binary',
       'G♯4_binary', 'A4_binary', 'A♯4_binary', 'B4_binary', 'C5_binary',
       'C♯5_binary', 'D5_binary', 'D♯5_binary', 'E5_binary', 'F5_binary',
       'F♯5_binary', 'G5_binary', 'G♯5_binary', 'A5_binary', 'A♯5_binary',
       'B5_binary', 'C6_binary', 'E2_freq', 'F2_freq', 'F♯2_freq', 'G2_freq',
       'G♯2_freq', 'A2_freq', 'A♯2_freq', 'B2_freq', 'C3_freq', 'C♯3_freq',
       'D3_freq', 'D♯3_freq', 'E3_freq', 'F3_freq', 'F♯3_freq', 'G3_freq',
       'G♯3_freq', 'A3_freq', 'A♯3_freq', 'B3_freq', 'C4_freq', 'C♯4_freq',
       'D4_freq', 'D♯4_freq', 'E4_freq', 'F4_freq', 'F♯4_freq', 'G4_freq',
       'G♯4_freq', 'A4_freq', 'A♯4_freq', 'B4_freq', 'C5_freq', 'C♯5_freq',
       'D5_freq', 'D♯5_freq', 'E5_freq', 'F5_freq', 'F♯5_freq', 'G5_freq',
       'G♯5_freq', 'A5_freq', 'A♯5_freq', 'B5_freq', 'C6_freq'])

In [12]:
music_pure_db['file'].unique()

array(['17さいのうた。 ⧸ 『ユイカ』【MV】', '213', 'AKB0048_-_Niji_no_Ressha',
       'AKB0048_Next_Stage_-_Aruji_Naki_Sono_Koe',
       'Akane-Iro_ni_Somaru_Saka_-_Mezamenai_Wish',
       'Akane-iro_ni_Somaru_Saka_-_Ai_Koko_Kara', 'Angel_Beats_-_Alchemy',
       'Angel_Beats_-_Crow_Song', 'Angel_Beats_-_Ichiban_no_Takaramono',
       'Angel_Beats_-_Ichiban_no_Takaramono_Karuta_Version',
       'AnoHana_-_Secret_Base_Kimi_ga_Kureta_Mono_10_Years_After',
       'Aoi_Hana_-_Centifolia', 'Aoi_Hana_-_Centifolia_2',
       'Bakemonogatari-Kimi no Shiranai Monogatari',
       'Bakemonogatari-Renai Circulation',
       'Bakemonogatari-SugarSweet Nightmare', 'Bleach_-_Change',
       'Bleach_-_Harukaze', 'Blood-C_-_Junketsu_Paradox',
       'Chuunibyou_Demo_Koi_ga_Shitai_-_Inside_Identity',
       'Chuunibyou_Demo_Koi_ga_Shitai_Lite_-_Shikkoku_ni_Odoru_Haou_Bushi',
       'Densetsu_no_Yuusha_no_Densetsu_-_Last_Inferno',
       'Ef_-_A_Tale_of_Melodies_-_Ebullient_Future',
       'End_of_Evangelion_-_Thanat

In [13]:
cache = music_pure_db[music_pure_db['file']=='AKB0048_Next_Stage_-_Aruji_Naki_Sono_Koe']
df = pd.DataFrame(cache['note'].value_counts().sort_index(key=lambda x: librosa.note_to_hz(x)))

binary = df.copy()
binary['count'].sum()*0.02

1172.8

In [14]:
for song in music_pure_db['file'].unique():
    # music_pure_db[music_pure_db['file']==song].to_csv(song+'.csv', index=False)
    cache = music_pure_db[music_pure_db['file']==song]
    df = pd.DataFrame(cache['note'].value_counts().sort_index(key=lambda x: librosa.note_to_hz(x)))
    
    binary = df.copy()
    binary.index = binary.index + "_binary"
    binary['count'] = binary['count'].apply(lambda x: 1 if x>binary['count'].sum()*0.02 else 0)
    
    freq = df.copy()
    freq.index = freq.index + "_freq"
    
    df = pd.concat([binary, freq])
    df = df.transpose()
    
    df['Name'] = song
    
    DB_music = pd.concat([DB_music, df], ignore_index=True)

DB_music

,Name,Album,Artist,Genre,Subgenre,Publish Date,Duration,BPM,Vocal Rage,E2_binary,...,F♯6_freq,G6_freq,A6_freq,B6_freq,E6_binary,G♯6_binary,A♯6_binary,E6_freq,G♯6_freq,A♯6_freq
0,17さいのうた。 ⧸ 『ユイカ』【MV】,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,2.0,2.0,2.0,842.0,NaN,NaN,NaN,NaN,NaN,NaN
2,AKB0048_-_Niji_no_Ressha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AKB0048_Next_Stage_-_Aruji_Naki_Sono_Koe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,26.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Akane-Iro_ni_Somaru_Saka_-_Mezamenai_Wish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,2.0,6.0,8.0,1032.0,0.0,0.0,0.0,4.0,2.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,ไม่เคย,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,4.0,22.0,NaN,0.0,NaN,0.0,4.0,NaN,32.0
212,ไม่เป็นรอง,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,1.0,7.0,12.0,876.0,0.0,0.0,0.0,3.0,27.0,8.0
213,スノードーム ⧸ 『ユイカ』【MV】,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214,好きだから。⧸ 『ユイカ』【MV】,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df

note,C1_binary,C♯1_binary,D1_binary,D♯1_binary,E1_binary,F1_binary,F♯1_binary,G1_binary,G♯1_binary,A1_binary,...,F5_freq,F♯5_freq,G5_freq,G♯5_freq,A5_freq,A♯5_freq,B5_freq,C6_freq,C♯6_freq,Name
count,0,0,0,0,0,0,0,0,0,0,...,82,8,14,72,2,12,2,122,8,恋泥棒。 ⧸ 『ユイカ』【MV】


In [16]:
DB_music.to_csv('DB_music_full.csv', index=False)

In [17]:
def detect_language(text):
    thai_range = (0x0E01, 0x0E5B)  # Thai unicode range
    # English unicode range (upper and lower case)
    english_range = (0x0041, 0x007A)

    langs_dict = {'Thai': 0, 'English': 0, "Numerical": 0, "Unknown": 0}

    # Check if the text contains Thai characters
    for char in text:
        if ord(char) >= thai_range[0] and ord(char) <= thai_range[1]:
            return "Thai"

    # Check if the text contains English characters
    for char in text:
        if (ord(char) >= english_range[0] and ord(char) <= english_range[1]) or (ord(char) >= english_range[0]+32 and ord(char) <= english_range[1]+32):
            return "English"

    if any(char.isdigit() for char in text):
        return "Numerical"

    return "Unknown"


DB_music[DB_music['Name'].map(lambda x: detect_language(x) == 'Thai' or str(x) in ['213'])].to_csv(
    'DB_music_only_Thai.csv', index=False)